### Import Required Package

In [1]:
import pandas as pd
import talib

import json

### Import Tidal

In [2]:
import tidal as td

### Initialize Plumber

In [3]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [4]:
import pandas as pd
import os
import numpy as np
os.chdir('/home/jovyan/individualized-indicator')
test_1 = pd.read_pickle('./emb/2023_03_01/top_8_2023-06-01_2024-06-28.pkl')
df_reset = test_1.reset_index()
unique_values, indices, counts = np.unique(df_reset.iloc[:,0], return_index=True, return_counts=True)
sorted_unique_values = unique_values[np.argsort(indices)]
modified_arr = np.array(sorted_unique_values)
# df_reset.iloc[:,0] = np.array([s[:-3] for s in df_reset.iloc[:,0]]) 
df_reset.iloc[:,0] = np.array(df_reset.iloc[:,0])
multi_index_df = df_reset.set_index(['instrument', 'datetime'])
multi_index_df = multi_index_df.dropna()

DEFAULT_STOCKS = list(modified_arr.astype(object))
start_date = "2023-06-01"
end_date = "2024-06-28"

quote_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in DEFAULT_STOCKS]),
    },
)
quote_data.index.set_levels(
    pd.to_datetime(quote_data.index.levels[1]),
    level=1,
    inplace=True,
)
quote_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

instruments = quote_data.index.get_level_values('instrument').unique()

quote_data['new'] = multi_index_df.iloc[:,5]

quote_data

/tmp/ipykernel_30444/3292540928.py:28: FutureWarning: inplace is deprecated and will be removed in a future version.
  quote_data.index.set_levels(


open      high       low     close   volume  \
instrument datetime                                                      
4743       2023-06-01  202.0706  203.3885  199.8742  200.3135  1394543   
           2023-06-02  202.0706  214.3706  201.6314  210.4170  5346699   
           2023-06-05  210.8563  213.4920  206.4635  207.3420  2717866   
           2023-06-06  208.6599  211.7349  206.9028  210.4170  1998439   
           2023-06-07  211.7349  213.4920  210.4170  210.8563  1515098   
...                         ...       ...       ...       ...      ...   
6829       2024-06-24  115.5000  118.0000  114.5000  117.0000   551628   
           2024-06-25  117.0000  118.0000  112.5000  115.0000   539068   
           2024-06-26  115.5000  117.0000  115.0000  116.0000   254166   
           2024-06-27  116.0000  117.5000  113.5000  116.5000   282997   
           2024-06-28  116.5000  117.5000  115.5000  117.0000   347439   

                            new  
instrument datetime              
4743       2023-06-01  0.188195  
           2023-06-02  0.254313  
           2023-06-05  0.279974  
           2023-06-06  0.267629  
           2023-06-07  0.256640  
...                         ...  
6829       2024-06-24  0.337327  
           2024-06-25  0.337085  
           2024-06-26  0.337317  
           2024-06-27  0.337363  
           2024-06-28  0.337511  

[93365 rows x 6 columns]

### Load benchmark data

In [5]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_30444/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2023-06-01  120.7304  120.9711  120.2969  120.5377   4231123
           2023-06-02  121.1638  122.3195  121.1638  122.0788   8412015
           2023-06-05  122.0788  122.1269  121.5972  121.6935   6813168
           2023-06-06  121.6935  122.5603  121.5009  122.0306   6877372
           2023-06-07  122.3195  123.5235  122.3195  123.3790  12723827
...                         ...       ...       ...       ...       ...
           2024-06-24  186.5000  186.5000  183.5500  184.0000  14671365
           2024-06-25  181.9500  183.9000  180.8000  183.9000  13774562
           2024-06-26  184.9500  186.1000  184.2500  185.3500   9269483
           2024-06-27  183.6000  185.3000  183.4000  185.3000   6508242
           2024-06-28  185.3000  187.4000  184.6500  186.4500   4455951

[263 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [6]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=10000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 2, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [7]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [8]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [9]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [10]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [11]:
tidal.backtest()

Tidal Backtesting: 100%|██████████| 263/263 [00:22<00:00, 11.83it/s, cash=2.06e+5, pnl=9.34e+5, position_cost=1.05e+7, value=1.17e+7] 


### Metric - AccountInfo

In [12]:
tidal.metrics['AccountInfo'].report

,cash,trade_cost,position_cost,slip_cost,pnl,value,max_drawdown
datetime,,,,,,,
2023-06-01,1.000000e+07,0.000000,0.000000e+00,0.0000,0.000000e+00,1.000000e+07,0.000000
2023-06-02,6.617157e+05,3059.619879,9.335225e+06,63613.6492,-7.545395e+04,9.921486e+06,-0.007851
2023-06-05,6.617157e+05,3059.619879,9.335225e+06,63613.6492,-4.170525e+04,9.955235e+06,-0.004476
2023-06-06,8.568877e+04,9864.567451,9.881429e+06,116991.9618,-1.864246e+05,9.780693e+06,-0.021931
2023-06-07,1.226261e+05,18202.545975,9.741403e+06,143815.6702,-6.757568e+04,9.796454e+06,-0.020355
...,...,...,...,...,...,...,...
2024-06-24,2.058344e+05,130570.749678,1.051810e+07,80885.4138,6.827286e+05,1.140666e+07,-0.073449
2024-06-25,2.058344e+05,130570.749678,1.051810e+07,80885.4138,7.760429e+05,1.149997e+07,-0.065869
2024-06-26,2.058344e+05,130570.749678,1.051810e+07,80885.4138,1.073674e+06,1.179761e+07,-0.041693


### Metric - PositionInfo

In [13]:
tidal.metrics['PositionInfo'].report

quantity       price  commission  slip_cost         pnl
instrument datetime                                                           
1308       2023-06-20   74000.0   25.495500  618.355109  -3529.800  -21201.000
           2023-06-21   74000.0   25.495500  618.355109  -3529.800  -14134.000
           2023-06-26   74000.0   25.495500  618.355109  -3529.800   -7067.000
           2023-06-27   74000.0   25.495500  618.355109  -3529.800    3529.800
           2023-06-28   74000.0   25.495500  618.355109  -3529.800       0.000
...                         ...         ...         ...        ...         ...
8996       2023-08-18    5000.0  348.503315  571.109808  23240.077 -215917.577
           2023-08-21    5000.0  348.503315  571.109808  23240.077 -218387.577
           2023-08-22    5000.0  348.503315  571.109808  23240.077 -213447.077
           2023-08-23    5000.0  348.503315  571.109808  23240.077 -210977.077
           2023-08-24    5000.0  348.503315  571.109808  23240.077 -186274.577

[1309 rows x 5 columns]

In [14]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl
instrument,datetime,,,,,


### Metric - AdditionalInfo

In [15]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2023-06-01  0.179093
           2023-06-02  0.236478
           2023-06-05  0.243812
           2023-06-06  0.239105
           2023-06-07  0.236817
...                         ...
9958       2024-06-24  0.333877
           2024-06-25  0.336709
           2024-06-26  0.339895
           2024-06-27  0.342637
           2024-06-28  0.345978

[93365 rows x 1 columns]

### Trade Report

In [16]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
2383,1.0,0.0,1.0,0.0,1.0,1.354342e+06,0.000000,1.129958e+04,1.354342e+06
8210,1.0,0.0,1.0,1.0,1.0,1.208260e+06,0.000000,3.005740e+06,1.208260e+06
2059,1.0,0.0,1.0,1.0,1.0,7.148812e+05,0.000000,1.765119e+06,7.148812e+05
3038,1.0,0.0,1.0,0.0,1.0,1.355981e+05,0.000000,6.900023e+03,1.355981e+05
8050,1.0,0.0,1.0,0.0,1.0,6.396989e+04,0.000000,7.057243e+03,6.396989e+04
2357,1.0,0.0,1.0,0.0,1.0,5.894308e+04,0.000000,7.859487e+03,5.894308e+04
2014,1.0,0.0,1.0,0.0,1.0,5.285692e+04,0.000000,7.219520e+03,5.285692e+04
2393,1.0,0.0,1.0,0.0,1.0,8.211823e+03,0.000000,8.242068e+03,8.211823e+03
1308,0.0,1.0,1.0,0.0,0.0,0.000000e+00,-3628.092681,6.907625e+03,-3.628093e+03


### Traded instruments

In [17]:
tidal.account.trades.keys()

dict_keys(['1565', '1326', '3035', '3037', '8050', '6747', '2393', '2014', '2912', '3038', '1308', '2383', '8996', '2376', '2059', '4979', '2755', '2357', '2301', '8210', '2385'])

### Trading History

In [18]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [19]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [20]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:41915
 * Running on http://10.136.1.95:41915
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:40] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:44] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:46] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:46] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:46] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:46] "GET /Tidal_Logo.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [09/Jul/2024 10:01:46] "GET 